In [21]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from streamlit.components.v1 import html
from pathlib import Path
import base64
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


# Configuración inicial de la página
st.set_page_config(
    layout="wide",
    page_title="Dashboard Irlanda",
    page_icon="🍀",
    initial_sidebar_state="expanded"
)

# Función para convertir imágenes a base64
def img_to_base64(img_path):
    try:
        with open(img_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode()
    except FileNotFoundError:
        st.error(f"Imagen no encontrada: {img_path}")
        return ""

# Función para cargar CSS
def load_css(file_name):
    try:
        with open(file_name, encoding="utf-8") as f:
            st.markdown(f"<style>{f.read()}</style>", unsafe_allow_html=True)
    except FileNotFoundError:
        st.error(f"Archivo CSS no encontrado: {file_name}")

# Cargar CSS primero
load_css("estil.css")

# Carga de datos con cache
@st.cache_resource
def load_data():
    try:
        df = pd.read_csv("Datos_limpios_Irlanda.csv", index_col='host_name')
        
        # Selección de columnas
        numeric_df = df.select_dtypes(['float', 'int'])
        numeric_cols = numeric_df.columns
        
        text_df = df.select_dtypes(['object'])
        text_cols = text_df.columns
        
        # Columnas categóricas relevantes
        categorical_column = df['host_is_superhost']
        unique_categories = categorical_column.unique()
        
        return df, numeric_cols, text_cols, unique_categories, numeric_df
    except Exception as e:
        st.error(f"Error al cargar los datos: {str(e)}")
        return pd.DataFrame(), [], [], [], pd.DataFrame()

# Cargar los datos
df, numeric_cols, text_cols, unique_categories, numeric_df = load_data()

# Menú de navegación mejorado
st.sidebar.markdown("""
<div style="text-align: center; margin-bottom: 20px;">
    <h2 style="color: #ffffff; margin-bottom: 5px;">🍀 Irlanda</h2>
    <p style="color: #e0e0e0; font-size: 14px;">Dashboard de análisis de Airbnb</p>
</div>
""", unsafe_allow_html=True)

View = st.sidebar.radio(
    "Menú de Navegación",
    options=[
        "🏠 Página Principal",
        "📊 Análisis de Frecuencias", 
        "📈 Diagramas de Dispersión",
        "📉 Regresión Lineal Simple",
        "📊 Regresión Lineal Múltiple",
    ],
    index=0,
    help="Selecciona el módulo que deseas visualizar"
)

# Mapeo de opciones
view_mapping = {
    "🏠 Página Principal": "home",
    "📊 Análisis de Frecuencias": "freq",
    "📈 Diagramas de Dispersión": "scatter",
    "📉 Regresión Lineal Simple": "linear",
    "📊 Regresión Lineal Múltiple": "multi",
}

View = view_mapping[View]

# Página Principal
if View == "home":
    try:
        # Leer y procesar el HTML
        html_content = Path("Presentacion.html").read_text(encoding="utf-8")
        
        # Reemplazar imágenes con base64
        html_content = html_content.replace(
            'src="logo.png"',
            f'src="data:image/png;base64,{img_to_base64("logo.png")}"'
        )
        
        # Mostrar el HTML
        st.components.v1.html(html_content, height=1000, scrolling=True)
        
    except Exception as e:
        st.error(f"Error al cargar la presentación: {str(e)}")
        st.markdown("""
        ## 🍀 Dashboard de Irlanda
        Explora los datos de Airbnb en Irlanda a través de nuestras herramientas de análisis.
        
        ### Características principales:
        - 📊 Análisis de distribuciones de precios
        - 📈 Visualización de relaciones entre variables
        - 📉 Modelos de regresión para predicción
        """)

# Análisis de Frecuencias
elif View == "freq":
    st.title("📊 Análisis de Frecuencias")
    st.markdown("""
    <style>
    .freq-header {
        background: linear-gradient(135deg, #2c7e3b 0%, #4CAF50 100%);
        color: white;
        padding: 15px;
        border-radius: 8px;
        margin-bottom: 25px;
    }
    </style>
    <div class="freq-header">
        <h3 style="color:white; margin:0;">Explora las distribuciones de las variables numéricas</h3>
    </div>
    """, unsafe_allow_html=True)
    
    # Widgets en sidebar
    st.sidebar.header("Configuración")
    numeric_var = st.sidebar.selectbox("Variable numérica:", numeric_cols, index=0)
    bins = st.sidebar.slider("Número de intervalos:", 3, 20, 5, 1)
    graph_type = st.sidebar.radio("Tipo de gráfico:", ["Barras", "Pastel", "Histograma"], index=0)
    show_raw = st.sidebar.checkbox("Mostrar datos originales")
    
    # Mostrar datos originales si está seleccionado
    if show_raw:
        with st.expander("📋 Datos Originales", expanded=False):
            st.subheader("Estadísticas Descriptivas")
            st.dataframe(df[[numeric_var]].describe().T.style.background_gradient(cmap='Greens'))
            st.subheader("Primeras 10 filas")
            st.dataframe(df[[numeric_var]].head(10))
    
    # Crear intervalos y tabla de frecuencias
    try:
        # Crear los intervalos
        df['Intervalo'] = pd.cut(df[numeric_var], bins=bins)
        
        # Crear tabla de frecuencias
        freq_table = df['Intervalo'].value_counts().sort_index().reset_index()
        freq_table.columns = ['Intervalo', 'Frecuencia Absoluta']
        freq_table['Frecuencia Relativa %'] = (freq_table['Frecuencia Absoluta'] / len(df)) * 100
        freq_table['Intervalo_str'] = freq_table['Intervalo'].astype(str)  # Convertir a string para visualización
        
        # Mostrar tabla de frecuencias
        st.subheader(f"Tabla de Frecuencias para: {numeric_var}")
        st.dataframe(
            freq_table[['Intervalo_str', 'Frecuencia Absoluta', 'Frecuencia Relativa %']]
            .rename(columns={'Intervalo_str': 'Intervalo'})
            .style
            .background_gradient(subset=['Frecuencia Absoluta'], cmap='Greens')
            .format({'Frecuencia Relativa %': '{:.2f}%'})
        )
        
        # Gráfico según selección
        st.subheader(f"Visualización: {graph_type}")
        
        if graph_type == "Barras":
            fig = px.bar(
                freq_table,
                x='Intervalo_str',
                y='Frecuencia Absoluta',
                color='Frecuencia Absoluta',
                color_continuous_scale='Greens',
                title=f"Distribución de {numeric_var} (Barras)",
                labels={'Intervalo_str': 'Intervalo'}
            )
            fig.update_xaxes(type='category')  # Para evitar problemas con los intervalos
            
        elif graph_type == "Pastel":
            fig = px.pie(
                freq_table,
                names='Intervalo_str',
                values='Frecuencia Absoluta',
                title=f"Distribución de {numeric_var} (Pastel)",
                color_discrete_sequence=px.colors.sequential.Greens_r,
                labels={'Intervalo_str': 'Intervalo'}
            )
            fig.update_traces(textposition='inside', textinfo='percent+label')
            
        else:  # Histograma
            fig = px.histogram(
                df,
                x=numeric_var,
                nbins=bins,
                title=f"Distribución de {numeric_var} (Histograma)",
                color_discrete_sequence=['#2e7d32']
            )
        
        st.plotly_chart(fig, use_container_width=True)
        
    except Exception as e:
        st.error(f"Error al generar la visualización: {str(e)}")
        st.info("Intenta ajustar el número de intervalos o seleccionar otra variable")

# Diagramas de Dispersión
elif View == "scatter":
    st.title("📈 Diagramas de Dispersión")
    st.markdown("""
    <style>
    .scatter-header {
        background-color: #0288d1;
        color: white;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 20px;
    }
    </style>
    <div class="scatter-header">
        Explora relaciones entre variables numéricas. Identifica correlaciones, patrones y valores atípicos.
    </div>
    """, unsafe_allow_html=True)
    
    # Selectores
    col1, col2 = st.columns(2)
    with col1:
        x_var = st.selectbox("Variable X", numeric_cols, index=0)
    with col2:
        y_var = st.selectbox("Variable Y", numeric_cols, index=1 if len(numeric_cols) > 1 else 0)
    
    # Opciones avanzadas
    with st.expander("⚙️ Opciones avanzadas"):
        color_by = st.selectbox("Color por", ['Ninguno'] + list(text_cols))
        trendline = st.checkbox("Mostrar línea de tendencia")
        log_x = st.checkbox("Escala logarítmica en X")
        log_y = st.checkbox("Escala logarítmica en Y")
    
    # Crear gráfico
    fig = px.scatter(
        df,
        x=x_var,
        y=y_var,
        color=color_by if color_by != 'Ninguno' else None,
        title=f"Relación entre {x_var} y {y_var}",
        trendline="ols" if trendline else None,
        log_x=log_x,
        log_y=log_y,
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    
    # Calcular correlación si son numéricas
    if x_var in numeric_cols and y_var in numeric_cols:
        corr = df[[x_var, y_var]].corr().iloc[0,1]
        st.markdown(f"**Coeficiente de correlación:** `{corr:.3f}`")
    
    st.plotly_chart(fig, use_container_width=True)





# Regresión Lineal Simple
elif View == "linear":
    st.title("📉 Regresión Lineal Simple")
    st.markdown("""
    <style>
    .linear-header {
        background: linear-gradient(135deg, #d32f2f 0%, #f44336 100%);
        color: white;
        padding: 15px;
        border-radius: 8px;
        margin-bottom: 25px;
    }
    </style>
    <div class="linear-header">
        <h3 style="color:white; margin:0;">Modelo predictivo con una variable independiente</h3>
    </div>
    """, unsafe_allow_html=True)

    if len(numeric_cols) < 2:
        st.warning("Se necesitan al menos 2 variables numéricas para realizar la regresión.")
    else:
        # Selección de variables
        col1, col2 = st.columns(2)
        with col1:
            x_var = st.selectbox("Variable independiente (X)", numeric_cols, index=0)
        with col2:
            y_var = st.selectbox("Variable dependiente (Y)", numeric_cols, index=1 if len(numeric_cols) > 1 else 0)

        # Filtrar y preparar datos
        model_df = df[[x_var, y_var]].dropna()
        if len(model_df) < 2:
            st.error("No hay suficientes datos para realizar la regresión después de eliminar valores faltantes.")
            st.stop()

        X = model_df[[x_var]]
        y = model_df[y_var]

        # Añadir constante para el intercepto
        X_const = sm.add_constant(X)
        model = sm.OLS(y, X_const).fit()

        # PESTAÑAS
        tab1, tab2, tab3 = st.tabs(["📊 Resumen del Modelo", "📈 Coeficientes", "🔮 Predicción"])

        with tab1:
            st.subheader("Métricas Clave del Modelo")
            col1, col2, col3 = st.columns(3)
            col1.metric("R²", f"{model.rsquared:.4f}")
            col2.metric("R² Ajustado", f"{model.rsquared_adj:.4f}")
            col3.metric("Observaciones", len(model_df))

            st.markdown(f"""
            **Otras estadísticas:**
            - **F-statistic:** {model.fvalue:.2f}
            - **Prob (F-statistic):** {model.f_pvalue:.4f}
            - **Log-Likelihood:** {model.llf:.2f}
            - **AIC:** {model.aic:.2f}
            - **BIC:** {model.bic:.2f}
            """)

            # Diagnóstico
            st.subheader("Diagnósticos del Modelo")
            diag_col1, diag_col2 = st.columns(2)

            try:
                dw = sm.stats.durbin_watson(model.resid)
                diag_col1.metric("Durbin-Watson", f"{dw:.3f}")
            except Exception:
                diag_col1.warning("No se pudo calcular Durbin-Watson")

            diag_col2.metric("Cond. No.", f"{model.condition_number:.2e}")
            if model.condition_number > 1000:
                st.warning(f"""
                **Nota:** El número de condición es alto ({model.condition_number:.2e}), lo que puede indicar:
                - Multicolinealidad
                - Variables con escalas muy diferentes
                - Problemas numéricos
                """)

        with tab2:
            st.subheader("Coeficientes del Modelo")
            coef_df = pd.DataFrame({
                "Variable": model.params.index,
                "Coeficiente": model.params.values,
                "Error Estándar": model.bse.values,
                "t-valor": model.tvalues.values,
                "p-valor": model.pvalues.values,
                "[0.025": model.conf_int()[0].values,
                "0.975]": model.conf_int()[1].values
            })

            coef_df = coef_df.round(6)

            st.dataframe(
                coef_df.style
                .background_gradient(subset=['t-valor'], cmap='Reds')
                .highlight_between(subset=['p-valor'], color='lightgreen', left=0, right=0.05)
                .set_properties(subset=['Variable'], **{'font-weight': 'bold'})
            )

            st.markdown("""
            **Interpretación:**
            - **Intercepto (const):** Valor esperado de Y cuando X = 0
            - **Coeficiente:** Cambio esperado en Y por cada unidad de cambio en X
            - **p-valor < 0.05:** Relación estadísticamente significativa
            """)

        with tab3:
            st.subheader("Realizar Predicción")
            x_value = st.number_input(
                f"Ingrese un valor para {x_var}:",
                min_value=float(X[x_var].min()),
                max_value=float(X[x_var].max()),
                value=float(X[x_var].mean())
            )

            if st.button("Calcular Predicción"):
                new_X = sm.add_constant(pd.DataFrame({x_var: [x_value]}))
                prediction = model.predict(new_X)[0]
                st.success(f"**Valor predicho para {y_var}:** {prediction:.4f}")

        # Visualización
        st.subheader("Visualización de la Regresión")
        fig = px.scatter(
            model_df,
            x=x_var,
            y=y_var,
            trendline="ols",
            trendline_color_override="#f44336",
            title=f"Regresión Lineal: {y_var} ~ {x_var}",
            color_discrete_sequence=['#d32f2f']
        )
        fig.update_layout(
            hovermode='x unified',
            showlegend=False
        )
        st.plotly_chart(fig, use_container_width=True)


        

# Regresión Lineal Múltiple
elif View == "multi":
    st.title("📊 Regresión Lineal Múltiple")
    st.markdown("""
    <style>
    .multi-header {
        background-color: #7b1fa2;
        color: white;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 20px;
    }
    </style>
    <div class="multi-header">
        Modelo de regresión lineal múltiple para predecir una variable en función de varias variables predictoras.
    </div>
    """, unsafe_allow_html=True)
    
    if len(numeric_cols) < 2:
        st.warning("Se necesitan al menos 2 variables numéricas para realizar la regresión múltiple.")
    else:
        # Selección de variables
        y_var = st.selectbox("Variable dependiente (Y)", numeric_cols)
        x_vars = st.multiselect("Variables independientes (X)", [x for x in numeric_cols if x != y_var])
        
        if not x_vars:
            st.warning("Seleccione al menos una variable independiente.")
        else:
            # Preparar datos
            model_df = df[[y_var] + x_vars].dropna()
            X = model_df[x_vars]
            y = model_df[y_var]
            
            # Estandarizar variables
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            # Ajustar modelo
            model = LinearRegression()
            model.fit(X_scaled, y)
            
            # Mostrar coeficientes
            st.subheader("Coeficientes del Modelo")
            coef_df = pd.DataFrame({
                "Variable": x_vars,
                "Coeficiente": model.coef_,
                "Importancia Absoluta": np.abs(model.coef_)
            }).sort_values("Importancia Absoluta", ascending=False)
            
            st.dataframe(coef_df.style.background_gradient(subset=['Importancia Absoluta'], cmap='Purples'))
            
            # Gráfico de importancia
            fig = px.bar(
                coef_df,
                x='Variable',
                y='Importancia Absoluta',
                color='Coeficiente',
                color_continuous_scale='Purples',
                title="Importancia de las Variables Predictoras"
            )
            st.plotly_chart(fig, use_container_width=True)
            
            # Predicción
            st.subheader("Realizar Predicción")
            input_values = {}
            cols = st.columns(2)
            for i, var in enumerate(x_vars):
                with cols[i % 2]:
                    input_values[var] = st.number_input(
                        f"Ingrese valor para {var}:",
                        min_value=float(df[var].min()),
                        max_value=float(df[var].max()),
                        value=float(df[var].mean())
                    )
            
            if st.button("Predecir"):
                # Estandarizar los valores de entrada
                input_df = pd.DataFrame([input_values])
                input_scaled = scaler.transform(input_df)
                
                # Realizar predicción
                prediction = model.predict(input_scaled)
                st.success(f"Valor predicho para {y_var}: {prediction[0]:.2f}")


Overwriting app.py
